### Importing Libraries 

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf 

#ignore warnings
import warnings
warnings.filterwarnings('ignore')


### Get data from Yahoo Finance

You can change the 2 stocks as well as the market with the tickers of your choice. For some metrics you will need to use except the stock itself, a market index as a benchmark. The most common practice for major stocks is to use the S&P 500 Index. 

At the end we will have a dataframe with the returns of the 2 assets and the market, since all the risk metrics are using the returns and not the raw values.

In [10]:
symbol_1 = 'AAPL'
symbol_2 = 'TSLA'
market = '^GSPC'

df_asset1 = yf.download(symbol_1, start='2023-01-01', end='2023-12-31')['Adj Close'].rename(symbol_1);
df_asset2 = yf.download(symbol_2, start='2023-01-01', end='2023-12-31')['Adj Close'].rename(symbol_2);
df_market = yf.download(market, start='2023-01-01', end='2023-12-31')['Close'].rename(market);
df_main = pd.merge(df_asset1, df_asset2, on='Date')
df_main = pd.merge(df_main, df_market, on='Date')
start_date = '2023-01-01'
end_date = '2023-12-31'
df_main = df_main[start_date:end_date]
df_returns = df_main.pct_change().dropna()
df_returns

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,AAPL,TSLA,^GSPC
Date,,,
2023-01-04,0.010314,0.051249,0.007539
2023-01-05,-0.010605,-0.029039,-0.011646
2023-01-06,0.036794,0.024651,0.022841
2023-01-09,0.004089,0.059349,-0.000768
2023-01-10,0.004456,-0.007681,0.006978
...,...,...,...
2023-12-22,-0.005547,-0.007701,0.001660
2023-12-26,-0.002841,0.016116,0.004232
2023-12-27,0.000518,0.018822,0.001430


## Standard Deviation

- AAPL (Apple Inc.): 0.0125
- TSLA (Tesla Inc.): 0.0331
- ^GSPC (S&P 500 Index): 0.0082

These values indicate the degree of variation or volatility in the returns of each stock or index. A higher standard deviation implies higher volatility, while a lower standard deviation suggests lower volatility. Investors often use standard deviation as a measure of risk, with higher values representing a potentially riskier investment.

In [13]:
df = df_returns.copy()
    
def calculate_standard_deviation(returns):
    return np.std(returns)

print(f'{symbol_1} has a Standard Deviation of {calculate_standard_deviation(df[symbol_1])}')
print(f'{symbol_2} has a Standard Deviation of {calculate_standard_deviation(df[symbol_2])}')
print(f'{market} has a Standard Deviation of {calculate_standard_deviation(df[market])}')

AAPL has a Standard Deviation of 0.01254478155112219
TSLA has a Standard Deviation of 0.03309894573475674
^GSPC has a Standard Deviation of 0.008241846331439819


## Beta

Beta measures the sensitivity of an investment's returns to market movements. A beta greater than 1 indicates higher volatility compared to the market, while a beta less than 1 suggests lower volatility.

The beta coefficient measures the sensitivity of a stock's returns to changes in the returns of a benchmark index. A beta greater than 1 indicates higher volatility compared to the benchmark, while a beta less than 1 suggests lower volatility. Here are the beta values you provided:

- AAPL (Apple Inc.) has a Beta of approximately 1.1045 using ^GSPC (S&P 500 Index) as the benchmark.
- TSLA (Tesla Inc.) has a Beta of approximately 2.2193 using ^GSPC as the benchmark.

These beta values suggest how much the returns of each stock tend to move in relation to the movements in the S&P 500 Index. A beta greater than 1 for TSLA indicates that TSLA is expected to be more volatile than the benchmark, while a beta close to 1 for AAPL suggests a similar level of volatility as the benchmark. Beta is a useful metric for investors to assess the risk and volatility of a stock relative to the broader market.

In [14]:
df = df_returns.copy()
    
def calculate_beta(returns, benchmark_returns):
    covariance = returns.cov(benchmark_returns)
    variance = benchmark_returns.var()
    beta = covariance / variance
    return beta

print(f'{symbol_1} has a Beta of {calculate_beta(df[symbol_1], df[market])} using {market} as the benchmark')
print(f'{symbol_2} has a Beta of {calculate_beta(df[symbol_2], df[market])} using {market} as the benchmark')


AAPL has a Beta of 1.1045140840353886 using ^GSPC as the benchmark
TSLA has a Beta of 2.219296862746592 using ^GSPC as the benchmark


## Maximum Drawdown:

Maximum Drawdown is a risk metric that measures the maximum loss from a peak to a trough of a portfolio's value during a specific period, usually expressed as a percentage. It provides insight into the worst-case scenario or the largest decline an investment experienced over a certain time frame.

By running the python code below, we can calculate the maximum drawdown of the three stocks.

AAPL (Apple Inc.): -0.3724 (37.24% maximum loss)
TSLA (Tesla Inc.): -0.6238 (62.38% maximum loss)
^GSPC (S&P 500 Index): -0.2039 (20.39% maximum loss)

These values represent the largest percentage decline in the value of each stock or index during a specific period. Lower maximum drawdown values are generally considered better, as they indicate lower potential losses during adverse market conditions.


In [15]:
df = df_returns.copy()
    
def calculate_max_drawdown(returns):
    wealth_index = np.cumprod(1 + returns)
    peak_index = np.argmax(wealth_index)
    trough_index = np.argmin(wealth_index[:peak_index + 1])

    max_drawdown = (wealth_index.iloc[trough_index] - wealth_index.iloc[peak_index]) / wealth_index.iloc[peak_index]
    return max_drawdown

print(f'{symbol_1} has a maximum drawdown of {calculate_max_drawdown(df[symbol_1])}')
print(f'{symbol_2} has a maximum drawdown of {calculate_max_drawdown(df[symbol_2])}')
print(f'{market} has a maximum drawdown of {calculate_max_drawdown(df[market])}')



AAPL has a maximum drawdown of -0.3724441566909941
TSLA has a maximum drawdown of -0.6238494657551136
^GSPC has a maximum drawdown of -0.2038843028608435


## Sharpe Ratio:
This ratio assesses the risk-adjusted return of an investment by considering its return in relation to its volatility. A higher Sharpe ratio indicates a better risk-adjusted performance.

The Sharpe ratio is a measure of risk-adjusted performance and indicates the excess return generated per unit of risk, with risk measured as the standard deviation of returns. Higher Sharpe ratios generally suggest better risk-adjusted performance. Here are the Sharpe ratios you provided:

- AAPL (Apple Inc.): 0.1463
- TSLA (Tesla Inc.): 0.1176
- ^GSPC (S&P 500 Index): 0.1118

These values represent the ratio of the average excess return over the risk-free rate to the standard deviation of returns. The higher the Sharpe ratio, the better the risk-adjusted performance. Investors often use the Sharpe ratio to compare the risk-adjusted returns of different investments.

In [16]:
df = df_returns.copy()
    
def calculate_sharpe_ratio(returns, risk_free_rate):
    average_return = np.mean(returns)
    volatility = np.std(returns)
    sharpe_ratio = (average_return - risk_free_rate) / volatility
    return sharpe_ratio

print(f'{symbol_1} has a Sharpe ratio of {calculate_sharpe_ratio(df[symbol_1], 0)}')
print(f'{symbol_2} has a Sharpe ratio of {calculate_sharpe_ratio(df[symbol_2], 0)}')
print(f'{market} has a Sharpe ratio of {calculate_sharpe_ratio(df[market], 0)}')

AAPL has a Sharpe ratio of 0.1462701765459904
TSLA has a Sharpe ratio of 0.11760865791443038
^GSPC has a Sharpe ratio of 0.11184268785503289


## Sortino Ratio: 
imilar to the Sharpe ratio, the Sortino ratio focuses on downside volatility, considering only the standard deviation of negative returns. It provides a measure of risk-adjusted return with a focus on downside risk.

The Sortino ratio is a variation of the Sharpe ratio that focuses on downside risk, using only the standard deviation of negative returns in the denominator:

The Sortino ratio is a measure of risk-adjusted performance similar to the Sharpe ratio, but it focuses on downside risk only, using the standard deviation of negative returns. Higher Sortino ratios generally indicate better risk-adjusted performance, specifically in terms of mitigating downside volatility. Here are the Sortino ratios you provided:

AAPL (Apple Inc.): 0.2703
TSLA (Tesla Inc.): 0.2198
^GSPC (S&P 500 Index): 0.2021
These values represent the ratio of the average excess return over the risk-free rate to the downside semi-deviation. A higher Sortino ratio suggests better risk-adjusted returns, particularly in terms of protecting against downside volatility. Investors often use the Sortino ratio to assess the performance of an investment while giving more emphasis to downside risk.

In [17]:
df = df_returns.copy()
    
def calculate_sortino_ratio(returns, risk_free_rate=0):
    downside_returns = np.minimum(returns - risk_free_rate, 0)
    downside_volatility = np.std(downside_returns)

    average_return = np.mean(returns - risk_free_rate)  # Adjust for risk-free rate

    if downside_volatility == 0:
        return np.inf  # Avoid division by zero

    sortino_ratio = average_return / downside_volatility
    return sortino_ratio

print(f'{symbol_1} has a Sortino ratio of {calculate_sortino_ratio(df[symbol_1])}')
print(f'{symbol_2} has a Sortino ratio of {calculate_sortino_ratio(df[symbol_2])}')
print(f'{market} has a Sortino ratio of {calculate_sortino_ratio(df[market])}')

AAPL has a Sortino ratio of 0.2703070004934494
TSLA has a Sortino ratio of 0.2198285494487039
^GSPC has a Sortino ratio of 0.20205301637520193


## Treynor Ratio: 
This ratio assesses the risk-adjusted performance of an investment by considering its excess return per unit of systematic risk (beta). It is especially useful for evaluating the performance of portfolios with respect to market risk.


The Treynor Ratio is another risk-adjusted performance measure similar to the Sharpe ratio. It's calculated by dividing the excess return over the risk-free rate by the beta of the investment, where beta represents the investment's sensitivity to market movements. Here's a Python code snippet to calculate the Treynor Ratio:

The Treynor ratio, in the context of comparing an individual stock's performance to a market index, is calculated as the excess return over the risk-free rate divided by the beta of the stock. It provides a measure of risk-adjusted performance, considering the stock's systematic risk relative to the market.

Here are the Treynor ratios you provided:

- AAPL (Apple Inc.): 0.00166 using ^GSPC (S&P 500 Index) as the market.
- TSLA (Tesla Inc.): 0.00175 using ^GSPC as the market.

These values suggest how well each stock compensates investors for the systematic risk it carries relative to the S&P 500 Index. A higher Treynor ratio indicates better risk-adjusted performance per unit of systematic risk. Investors often use the Treynor ratio to assess the relative performance of individual stocks compared to a market benchmark.

In [18]:
df = df_returns.copy()
    
def calculate_treynor_ratio(returns, market_returns, risk_free_rate):
    excess_return = returns - risk_free_rate
    beta = calculate_beta(returns, market_returns)

    if beta == 0:
        return np.inf  # Avoid division by zero

    treynor_ratio = excess_return.mean() / beta
    return treynor_ratio

print(f'{symbol_1} has a Treynor ratio of {calculate_treynor_ratio(df[symbol_1], df[market], 0)} used {market} as the market')
print(f'{symbol_2} has a Treynor ratio of {calculate_treynor_ratio(df[symbol_2], df[market], 0)} used {market} as the market')

AAPL has a Treynor ratio of 0.0016612983380977283 used ^GSPC as the market
TSLA has a Treynor ratio of 0.0017540341950601794 used ^GSPC as the market


## Calmar Ratio:
The Calmar ratio is the ratio of the average annual rate of return to the maximum drawdown. It provides a measure of risk-adjusted return, emphasizing downside risk.


The Calmar Ratio is a risk-adjusted performance measure commonly used in the context of hedge funds and managed futures. It is calculated by dividing the average annualized return by the maximum drawdown, which represents the largest peak-to-trough decline in the value of a portfolio.

The Calmar Ratio is a risk-adjusted performance measure that evaluates the ratio of the average annual rate of return to the maximum drawdown, providing a measure of risk-adjusted returns. Higher Calmar ratios generally indicate better risk-adjusted performance. Here are the Calmar ratios you provided:

- AAPL (Apple Inc.): 1.2415
- TSLA (Tesla Inc.): 1.5724
- ^GSPC (S&P 500 Index): 1.1393

These values represent the ratio of the average annual rate of return to the maximum drawdown for each investment. A higher Calmar ratio suggests better risk-adjusted returns, considering the magnitude of the maximum drawdown. Investors often use the Calmar ratio to assess the trade-off between returns and drawdowns, with higher values indicating better risk-adjusted performance.

In [19]:
df = df_returns.copy()
    
def calculate_calmar_ratio(returns, period=252):  # Assuming 252 trading days in a year
    annualized_return = np.mean(returns) * period
    max_drawdown = calculate_max_drawdown(returns)

    if max_drawdown == 0:
        return np.inf  # Avoid division by zero

    calmar_ratio = annualized_return / abs(max_drawdown)
    return calmar_ratio

print(f'{symbol_1} has a Calmar ratio of {calculate_calmar_ratio(df[symbol_1])}')
print(f'{symbol_2} has a Calmar ratio of {calculate_calmar_ratio(df[symbol_2])}')
print(f'{market} has a Calmar ratio of {calculate_calmar_ratio(df[market])}')

AAPL has a Calmar ratio of 1.241532991109455
TSLA has a Calmar ratio of 1.5724403811854653
^GSPC has a Calmar ratio of 1.1393282311725124


## Ulcer Index:
This index quantifies the depth and duration of drawdowns in an investment's value. It provides a more comprehensive view of downside risk compared to maximum drawdown alone.

The Ulcer Index is a measure of risk that incorporates both the depth and duration of drawdowns in an investment. It is calculated by taking the square root of the average of the squared percentage drawdowns over a specified period.

The Ulcer Index is a measure of the depth and duration of drawdowns in an investment. It quantifies the extent of the price decline from its most recent peak, with lower values indicating less severe and shorter drawdowns. Here are the Ulcer Index values you provided:

- AAPL (Apple Inc.): 0.0551
- TSLA (Tesla Inc.): 0.1492
- ^GSPC (S&P 500 Index): 0.0341

These values represent the proportionate drawdown in each investment. A lower Ulcer Index suggests that the investment experiences less severe and shorter drawdowns, indicating potentially smoother performance. Investors use the Ulcer Index to assess the downside risk and volatility of an investment.

In [20]:
df = df_returns.copy()

def calculate_ulcer_index(returns):
    returns = returns.dropna()
    wealth_index = np.cumprod(1 + returns)
    previous_peaks = np.maximum.accumulate(wealth_index)
    drawdowns = (wealth_index - previous_peaks) / previous_peaks
    squared_drawdowns = np.square(drawdowns)
    ulcer_index = np.sqrt(np.mean(squared_drawdowns))
    return ulcer_index

print(f'{symbol_1} has an Ulcer index of {calculate_ulcer_index(df[symbol_1])}')
print(f'{symbol_2} has an Ulcer index of {calculate_ulcer_index(df[symbol_2])}')
print(f'{market} has an Ulcer index of {calculate_ulcer_index(df[market])}')



AAPL has an Ulcer index of 0.05509672077820546
TSLA has an Ulcer index of 0.14917579484097326
^GSPC has an Ulcer index of 0.03408355669692506


## Value at Risk (VaR):
VaR measures the maximum potential loss of an investment over a specific time period at a given confidence level. It helps investors understand the potential downside risk.

Value at Risk (VaR) is a statistical measure used to quantify the potential loss on an investment over a specific time horizon and with a certain confidence level. There are different methods to calculate VaR, and one common approach is the historical simulation method. Here's a simple Python code snippet using historical simulation to calculate VaR:

The Value at Risk (VaR) is a statistical measure that quantifies the maximum potential loss of an investment within a specific time frame and at a certain confidence level. Here are the VaR values you provided:

- AAPL (Apple Inc.): 0.0173
- TSLA (Tesla Inc.): 0.0503
- ^GSPC (S&P 500 Index): 0.0138

These values represent the estimated maximum potential loss, typically expressed as a percentage, over the specified time frame and confidence level. VaR is commonly used by investors and risk managers to assess the potential downside risk of an investment. A higher VaR indicates a higher level of potential loss.

In [21]:
df = df_returns.copy()
    
def calculate_var(returns, confidence_level=0.95):
    returns_sorted = np.sort(returns)
    n = len(returns)
    position = int(n * (1 - confidence_level))
    
    var = -returns_sorted[position]
    return var

print(f'{symbol_1} has a Value at Risk of {calculate_var(df[symbol_1])}')
print(f'{symbol_2} has a Value at Risk of {calculate_var(df[symbol_2])}')
print(f'{market} has a Value at Risk of {calculate_var(df[market])}')

AAPL has a Value at Risk of 0.01725376628542985
TSLA has a Value at Risk of 0.05030873801899527
^GSPC has a Value at Risk of 0.013788741489130674


## Conditional Value at Risk (CVaR):

Also known as Expected Shortfall, CVaR measures the expected loss beyond a certain confidence level. It provides insights into the severity of extreme losses.

Conditional Value at Risk (CVaR), also known as Expected Shortfall (ES), is a risk measure that quantifies the expected loss beyond the Value at Risk (VaR) at a certain confidence level. While VaR gives the maximum loss with a certain probability, CVaR provides an estimate of the expected loss given that the loss exceeds the VaR.

Conditional Value at Risk (CVaR), also known as Expected Shortfall (ES), is a risk measure that provides an estimate of the expected loss beyond the Value at Risk (VaR) at a certain confidence level. Here are the CVaR values you provided:

- AAPL (Apple Inc.): 0.0256
- TSLA (Tesla Inc.): 0.0667
- ^GSPC (S&P 500 Index): 0.0158

These values represent the expected loss beyond the VaR at a specified confidence level. CVaR is often considered a more informative risk measure than VaR, as it takes into account the severity of losses beyond the VaR threshold. A higher CVaR indicates a potentially greater expected loss beyond the VaR. Investors use CVaR to gain insights into the tail risk of an investment or portfolio.

In [22]:
df = df_returns.copy()
    
def calculate_cvar(returns, confidence_level=0.95):
    sorted_returns = np.sort(returns)
    n = len(returns)
    position = int(n * (1 - confidence_level))

    cvar = -np.mean(sorted_returns[:position])
    return cvar

print(f'{symbol_1} has a Conditional Value at Risk of {calculate_cvar(df[symbol_1])}')
print(f'{symbol_2} has a Conditional Value at Risk of {calculate_cvar(df[symbol_2])}')
print(f'{market} has a Conditional Value at Risk of {calculate_cvar(df[market])}')

AAPL has a Conditional Value at Risk of 0.02563116524184404
TSLA has a Conditional Value at Risk of 0.06672154043052804
^GSPC has a Conditional Value at Risk of 0.015845102985801485


## Downside Deviation:
Similar to standard deviation, downside deviation focuses on the variability of negative returns. It provides a more targeted measure of risk


Downside Deviation is a risk metric that focuses on the volatility of negative returns or downside risk. It measures the dispersion of returns below a certain threshold, typically zero. Here's a Python code snippet to calculate Downside Deviation:

Downside Deviation is a risk metric that focuses on the volatility of negative returns or downside risk. It measures the dispersion of returns below a certain threshold, typically zero. Here are the Downside Deviation values you provided:

AAPL (Apple Inc.): 0.0068
TSLA (Tesla Inc.): 0.0177
^GSPC (S&P 500 Index): 0.0046
These values represent the standard deviation of the negative returns, providing a measure of the volatility of downside risk. A higher Downside Deviation indicates greater volatility in negative returns, implying potentially larger and more frequent downside movements in the investment. Investors use Downside Deviation to assess the risk associated with negative returns and the potential for losses in adverse market conditions.

In [64]:
df = df_returns.copy()
    
def calculate_downside_deviation(returns, threshold=0):
    downside_returns = np.minimum(returns - threshold, 0)
    downside_deviation = np.std(downside_returns)
    return downside_deviation

print(f'{symbol_1} has a Downside Deviation of {calculate_downside_deviation(df[symbol_1])}')
print(f'{symbol_2} has a Downside Deviation of {calculate_downside_deviation(df[symbol_2])}')
print(f'{market} has a Downside Deviation of {calculate_downside_deviation(df[market])}')

AAPL has a Downside Deviation of 0.0067883086479370355
TSLA has a Downside Deviation of 0.01770799377974175
^GSPC has a Downside Deviation of 0.004562120690564971


## R-Squared:
R-squared measures the proportion of an investment's movement that can be explained by movements in its benchmark index. A higher R-squared indicates a closer correlation.

R-squared, also known as the coefficient of determination, is a statistical measure that represents the proportion of the variance in the dependent variable that is explained by the independent variable(s) in a regression model. In the context of financial analysis, it can be used to assess how well the performance of an investment or portfolio is explained by a benchmark index or other factors.

The R-squared value measures the proportion of the variance in the dependent variable (stock returns in this case) that is explained by the independent variable (benchmark returns, ^GSPC in this case) in a linear regression model. Here are the R-squared values you provided:

- AAPL (Apple Inc.): 0.5166 using ^GSPC as the benchmark.
- TSLA (Tesla Inc.): 0.2052 using ^GSPC as the benchmark.

These values represent the goodness of fit of the linear regression model. A higher R-squared value indicates a stronger linear relationship between the stock returns and the benchmark returns. In this context, R-squared is often used to assess how well the stock's performance aligns with movements in the benchmark index. Keep in mind that R-squared does not imply causation but only measures the strength and direction of the linear relationship.


In [23]:
df = df_returns.copy()
    
def calculate_r_squared(actual, predicted):
    mean_actual = np.mean(actual)
    total_sum_of_squares = np.sum((actual - mean_actual) ** 2)
    residual_sum_of_squares = np.sum((actual - predicted) ** 2)
    
    r_squared = 1 - (residual_sum_of_squares / total_sum_of_squares)
    return r_squared

print(f'{symbol_1} has a R-Squared of {calculate_r_squared(df[symbol_1], df[market])} using {market} as the benchmark index')
print(f'{symbol_2} has a R-Squared of {calculate_r_squared(df[symbol_2], df[market])} using {market} as the benchmark index')



AAPL has a R-Squared of 0.5165678698098413 using ^GSPC as the benchmark index
TSLA has a R-Squared of 0.20515035436500972 using ^GSPC as the benchmark index
